<a href="https://colab.research.google.com/github/gaveter/Project_cs/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Project 2019-20
1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.
3. Find the lenders that have funded at least twice.
4. For each country, compute how many loans have involved that country as borrowers.
5. For each country, compute the overall amount of money borrowed.
6. Like the previous point, but expressed as a percentage of the overall amount lent.
7. Like the three previous points, but split for each year (with respect to disburse time).
8. For each lender, compute the overall amount of money lent.
9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
10. Which country has the highest ratio between the difference computed at the previous point and the population?
11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must 
have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016,
 planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04
for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.
© 2019 GitHub, Inc.

## Load libraries

In [0]:
import pandas as pd
import datetime
import re
import numpy as np

In [7]:
loans_lender.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


## Task1

### Load csv

In [0]:
loans_lender = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans_lenders.csv')

In [0]:
loans_lender["lenders"]=loans_lender["lenders"].str.split(',') #divisione lenders
loans_lender_norm=loans_lender.explode('lenders')

### Result

In [9]:
loans_lender_norm=loans_lender_norm.reset_index(drop=True)
loans_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


## Task2
- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time -

In [0]:
loan = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans.csv')

In [16]:
loan.dtypes

loan_id                              int64
loan_name                           object
original_language                   object
description                         object
description_translated              object
funded_amount                      float64
loan_amount                        float64
status                              object
activity_name                       object
sector_name                         object
loan_use                            object
country_code                        object
country_name                        object
town_name                           object
currency_policy                     object
currency_exchange_coverage_rate    float64
currency                            object
partner_id                         float64
posted_time                         object
planned_expiration_time             object
disburse_time                       object
raised_time                         object
lender_term                        float64
num_lenders

In [0]:
prova = loan.head()
prova = prova[["planned_expiration_time","disburse_time"]] #scadenza, inizio_prestito

In [18]:
prova

,planned_expiration_time,disburse_time
0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000
1,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000
2,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000
3,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000
4,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000
